## Download images from flickr
Selenium to render the pages where requests could not!

In [1]:
from bs4 import BeautifulSoup
import requests 
import selenium
import time
from selenium import webdriver
import os

# for handling threads
import concurrent.futures

In [2]:
# Site runs JS so selenium to render the Javascript
driver = webdriver.Firefox()

In [3]:
base_url='https://www.flickr.com'
page_url_mask='https://www.flickr.com/photos/nasacommons/page'

cwd=os.getcwd()
data_folder=os.path.join(cwd,"Images")
if not os.path.exists(data_folder):
    os.makedirs(data_folder)

In [4]:
headers = {
    "Connection": "keep-alive",
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:91.0) Gecko/20100101 Firefox/91.0"
}
def download_image(img_url):
    #print(img_url)
    img_url=img_url+'sizes/l/'
    try:
        res=requests.get(img_url,allow_redirects = True,headers=headers)
        s1=BeautifulSoup(res.text)
        mya = s1.find_all("a")
        for e in mya:
            if 'Download' in e.text:
                img_url=e['href']
                img_name=e['href'].split("/")[-1]
                break
        img_bytes= requests.get(img_url).content
        with open(os.path.join(data_folder,img_name),"wb") as img_file:
            img_file.write(img_bytes)
            #print(f'file {img_name} downloaded..')
    except Exception as e:
        print(e)

In [5]:
LAST_PAGE=34
for page_no in range(1,LAST_PAGE+1):
    page_url=page_url_mask+str(page_no)
    driver.get(page_url)
    time.sleep(4)
    soup=BeautifulSoup(driver.page_source)
    images=soup.find_all("a", {"class": "overlay"})
    images_on_a_page=[]
    for each in images:
        images_on_a_page.append(base_url+each['href'])    
        
    with concurrent.futures.ThreadPoolExecutor() as executor:
        executor.map(download_image,images_on_a_page)
    
    #break
    

In [7]:
driver.close()

<bound method WebDriver.close of <selenium.webdriver.firefox.webdriver.WebDriver (session="97920b42-0703-914b-bc8f-74c67d38cb7e")>>

In [8]:
headers = {
    "Connection": "keep-alive",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3626.121 Safari/537.36"
}

In [9]:
res1=requests.get('https://www.flickr.com/photos/nasacommons/',allow_redirects = True,headers=headers)

In [10]:
s1=BeautifulSoup(res1.text)

In [42]:
import re
mya = s1.find_all("a",attrs={'data-track': re.compile('pagination[0-9]+')})

In [47]:
LAST_PAGE=1
for each_pagination in mya:
    if int(each_pagination.text) >= LAST_PAGE:
        LAST_PAGE=int(each_pagination.text)
print(f"Last page in page counter is {LAST_PAGE}")

Last page in page counter is 34


In [54]:
with open("test.html","w") as file:
    file.write(res.text)